In [2]:
import pandas as pd

import importlib

import src.valid_index
importlib.reload(src.valid_index)
from src.valid_index import get_excon_manual_index

import src.file_tools
importlib.reload(src.file_tools)
from src.file_tools import read_processed_regs_into_dataframe, get_regulation_detail, num_tokens_from_string

import src.embeddings
importlib.reload(src.embeddings)
from src.embeddings import get_ada_embedding

import src.tree_tools
importlib.reload(src.tree_tools)
from src.tree_tools import build_tree_for_regulation, split_tree

import src.summarise_and_question
importlib.reload(src.summarise_and_question)
from src.summarise_and_question import get_summary_and_questions_for


In [19]:
path_to_index_as_parquet_file = "./inputs/ad_index.parquet"
df_index = pd.read_parquet(path_to_index_as_parquet_file)
df_index


,section,text,Embedding,source
0,Legal context,Legal context,"[0.008710513822734356, -0.01103331707417965, 0...",heading
1,A.2(A),Authorised entities. Authorised Dealers,"[-0.009998462162911892, -0.009398686699569225,...",heading
2,A.2(B),Authorised entities. Authorised Dealers in for...,"[-0.009065092541277409, -0.012516901828348637,...",heading
3,A.3(A),Duties and responsibilities of Authorised Deal...,"[0.009497863240540028, -0.01100919209420681, -...",heading
4,A.3(B),Duties and responsibilities of Authorised Deal...,"[-0.010621572844684124, -0.014073583297431469,...",heading
...,...,...,...,...
1491,J.(E),What are the reporting requirements for cross-...,"[0.0023427396081387997, -0.022300338372588158,...",question
1492,J.(E),Requirement to accurately report all cross-bor...,"[-0.02419387362897396, -0.011988988146185875, ...",summary
1493,J.(G),What is the procedure for reporting outward pa...,"[-0.0016078203916549683, -0.000447303900728002...",question
1494,J.(G),Rules for reporting outward payments using the...,"[0.005922725424170494, -0.004037913400679827, ...",summary


In [23]:
df_tmp = df_index[df_index["section"]=="B.4(B)(ii)"]
for index, row in df_tmp.iterrows():
    print(f"-- {row['source']}: {row['text']}")

-- heading: Single discretionary allowance and other miscellaneous payments for private individuals. Travel allowances. Travel allowance limits
-- summary: Foreign currency can be provided within a single discretionary allowance limit of R1 million per calendar year to eligible parties, as long as certain specified requirements are met. However, residents under the age of 18 may receive a travel allowance of up to R200,000 per calendar year.
-- question: Who is eligible for a discretionary allowance limit in foreign currency?
-- question: What is the maximum amount I can exchange within a calendar year?


In [27]:
section = "B.4(B)(ii)"
text = "What are the travel allowance limits?"
embedding = get_ada_embedding(text)
source = "manual"

In [28]:
index = len(df_index)
index
df_index.loc[index] = [section, text, embedding, source]

In [26]:
df_index

,section,text,Embedding,source
0,Legal context,Legal context,"[0.008710513822734356, -0.01103331707417965, 0...",heading
1,A.2(A),Authorised entities. Authorised Dealers,"[-0.009998462162911892, -0.009398686699569225,...",heading
2,A.2(B),Authorised entities. Authorised Dealers in for...,"[-0.009065092541277409, -0.012516901828348637,...",heading
3,A.3(A),Duties and responsibilities of Authorised Deal...,"[0.009497863240540028, -0.01100919209420681, -...",heading
4,A.3(B),Duties and responsibilities of Authorised Deal...,"[-0.010621572844684124, -0.014073583297431469,...",heading
...,...,...,...,...
1492,J.(E),Requirement to accurately report all cross-bor...,"[-0.02419387362897396, -0.011988988146185875, ...",summary
1493,J.(G),What is the procedure for reporting outward pa...,"[-0.0016078203916549683, -0.000447303900728002...",question
1494,J.(G),Rules for reporting outward payments using the...,"[0.005922725424170494, -0.004037913400679827, ...",summary
1495,B.4(G)(i),How much money can an individual take offshore?,"[0.01233571209013462, -0.03362925723195076, 0....",manual


In [29]:
df_index.to_parquet(path_to_index_as_parquet_file)